In [46]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('/Users/andrewdrabkin/Documents/BLMA/monthly_realtor_metrics_combined.csv')

# Display the first few rows
print(df.head())


    zcta    date  median_listing_price  active_listing_count  \
0   8108  201607              230000.0                  97.0   
1  56368  201607              235000.0                  36.0   
2  46615  201607               70000.0                  79.0   
3  84061  201607             1280000.0                   2.0   
4  37151  201607              170000.0                   4.0   

   median_days_on_market  total_listing_count  new_listing_count  \
0                   62.0                146.0               36.0   
1                   67.0                 50.0               12.0   
2                   53.0                 79.0               36.0   
3                  218.0                  3.0                0.0   
4                   78.0                  4.0                0.0   

   price_drops_pct_of_inventory  price_reduced_count  average_listing_price  \
0                          0.45                 44.0               251000.0   
1                          0.11                 

In [47]:
#every instance has a zipcode
df['zcta'].isna().sum()

np.int64(0)

In [48]:
#how many zip codes have 4 numbers
(df['zcta'] < 10000).sum()

np.int64(253851)

In [49]:
df.shape

(3114037, 16)

In [50]:
print(df['date'].min())
print(df['date'].max())

201607
202508


Any zipcodes with four digits I added a zero before it to make it valid

In [51]:
# Convert to string first
df['zcta'] = df['zcta'].astype(str)

# add with zeros on the left if the string has less than 5 digits
df['zcta'] = df['zcta'].str.zfill(5)


invalid_zips = df[df['zcta'].str.len() != 5]

# Show how many there are
print(len(invalid_zips))

0


In [52]:
# Calculate percent of missing values in each column
na_percent = (df.isna().sum() / len(df)) * 100


na_percent.sort_values(ascending=False)


median_days_on_market_yy_smoothed       15.265169
median_listing_price_yy_smoothed        14.760293
average_listing_price_yy_smoothed       14.760293
total_listing_count_yy_smoothed         14.704482
price_drops_pct_of_inventory             1.441569
median_listing_price_per_square_foot     0.965306
median_days_on_market                    0.938139
median_listing_price                     0.246754
average_listing_price                    0.246754
active_listing_count                     0.151957
total_listing_count                      0.105362
new_listing_count                        0.105362
price_reduced_count                      0.105362
active_listing_count_historical          0.000417
zcta                                     0.000000
date                                     0.000000
dtype: float64

No Column has more than 20% na's

In [53]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

formula = 'median_listing_price ~ date'
model = ols(formula, data=df).fit()
print(model.summary())

                             OLS Regression Results                             
Dep. Variable:     median_listing_price   R-squared:                       0.009
Model:                              OLS   Adj. R-squared:                  0.009
Method:                   Least Squares   F-statistic:                 2.889e+04
Date:                  Sat, 08 Nov 2025   Prob (F-statistic):               0.00
Time:                          22:12:28   Log-Likelihood:            -4.6647e+07
No. Observations:               3106353   AIC:                         9.329e+07
Df Residuals:                   3106351   BIC:                         9.329e+07
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -5.803e+07   3.44e+05

In [54]:
# correlation graph 


corr = df.corr(numeric_only=True)["median_listing_price"].sort_values(ascending=False)
print(corr)


median_listing_price                    1.000000
average_listing_price                   0.912390
median_listing_price_per_square_foot    0.115055
date                                    0.095999
active_listing_count_historical         0.058867
total_listing_count                     0.040854
new_listing_count                       0.040021
active_listing_count                    0.038380
price_reduced_count                     0.022232
median_days_on_market_yy_smoothed       0.003824
median_listing_price_yy_smoothed        0.001714
average_listing_price_yy_smoothed       0.001534
price_drops_pct_of_inventory            0.000295
median_days_on_market                  -0.001251
total_listing_count_yy_smoothed        -0.007327
Name: median_listing_price, dtype: float64


In [55]:
model = ols('median_listing_price ~ median_days_on_market', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(model.summary())  # regression coefficients
     # ANOVA version of the same model







                             OLS Regression Results                             
Dep. Variable:     median_listing_price   R-squared:                       0.000
Model:                              OLS   Adj. R-squared:                  0.000
Method:                   Least Squares   F-statistic:                     4.830
Date:                  Sat, 08 Nov 2025   Prob (F-statistic):             0.0280
Time:                          22:12:31   Log-Likelihood:            -4.6278e+07
No. Observations:               3084398   AIC:                         9.256e+07
Df Residuals:                   3084396   BIC:                         9.256e+07
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept 

I' going to drop the following columns because they have a high amount of na's and are low correlation "median_listing_price_yy_smoothed",
    "average_listing_price_yy_smoothed",
    "total_listing_count_yy_smoothed",
    "median_days_on_market_yy_smoothed"

['zcta', 'date', 'median_listing_price', 'active_listing_count', 'median_days_on_market', 'total_listing_count', 'new_listing_count', 'price_drops_pct_of_inventory', 'price_reduced_count', 'average_listing_price', 'median_listing_price_per_square_foot', 'active_listing_count_historical', 'total_listing_count_yy_smoothed', 'median_days_on_market_yy_smoothed', 'median_listing_price_yy_smoothed', 'average_listing_price_yy_smoothed']


,zcta,date,median_listing_price,active_listing_count,median_days_on_market,total_listing_count,new_listing_count,price_drops_pct_of_inventory,price_reduced_count,average_listing_price,median_listing_price_per_square_foot,active_listing_count_historical
0,08108,201607,230000.0,97.0,62.0,146.0,36.0,0.45,44.0,251000.0,156.0,26.0
1,56368,201607,235000.0,36.0,67.0,50.0,12.0,0.11,4.0,249000.0,151.0,13.0
2,46615,201607,70000.0,79.0,53.0,79.0,36.0,0.25,20.0,83000.0,56.0,27.0
3,84061,201607,1280000.0,2.0,218.0,3.0,0.0,0.00,0.0,1280000.0,471.0,3.0
4,37151,201607,170000.0,4.0,78.0,4.0,0.0,0.00,0.0,177000.0,73.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3114032,07848,202508,574000.0,6.0,19.0,14.0,4.0,0.00,0.0,558000.0,240.0,11.0
3114033,96025,202508,344000.0,24.0,90.0,31.0,4.0,0.17,4.0,360000.0,232.0,14.0
3114034,14744,202508,215000.0,3.0,57.0,3.0,0.0,0.00,0.0,290000.0,143.0,2.0
3114035,53225,202508,249000.0,38.0,23.0,44.0,24.0,0.21,8.0,257000.0,179.0,37.0
